# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
pip install holoviews geoviews

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,castelnovo ne'monti,44.4356,10.4033,9.02,87,11,0.62,IT,1699835628
1,1,bloemhof,-27.6469,25.6070,18.58,32,2,3.22,ZA,1699835628
2,2,minas de marcona,-15.2119,-75.1103,14.62,91,90,6.45,PE,1699835628
3,3,bethel,41.3712,-73.4140,2.93,72,0,1.54,US,1699835562
4,4,alofi,-19.0595,-169.9187,25.94,73,100,6.69,NU,1699835628


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng",
                                     "Lat",
                                     geo = True,
                                     color = "City",
                                     alpha = 0.6,
                                     size = "Humidity",
                                     tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria
# Max Temp < 27 but > 21
# Cloudiness < 0
# Wind speed < 4.5 m/s

weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"]<27
                                       & (city_data_df["Max Temp"]>21)
                                       & (city_data_df["Cloudiness"]<0)
                                       & (city_data_df["Wind Speed"]<4.5))
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,labytnangi,66.6572,66.4183,-11.90,88,16,4.00,RU,1699835630
12,12,dudinka,69.4058,86.1778,-0.74,98,100,7.60,RU,1699835055
16,16,anadyr,64.7500,177.4833,-0.72,80,75,9.00,RU,1699835630
23,23,st. john's,47.5649,-52.7093,-0.02,87,100,3.09,CA,1699835338
26,26,yellowknife,62.4560,-114.3525,-6.99,95,100,8.75,CA,1699835632
29,29,boguchany,58.3814,97.4531,-9.52,94,100,1.34,RU,1699835632
30,30,longyearbyen,78.2186,15.6401,-4.09,80,75,7.72,SJ,1699835632
31,31,laojunmiao,39.8333,97.7333,-2.06,41,4,2.11,CN,1699835040
37,37,magadan,59.5667,150.8000,-10.26,61,100,3.75,RU,1699835634
47,47,happy valley-goose bay,53.3168,-60.3315,-2.83,68,75,4.63,CA,1699835412


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called 
# hotel_df to store the city, country, coordinates, and humidity

hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng",
                               "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can 
# store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
11,labytnangi,RU,66.6572,66.4183,88,
12,dudinka,RU,69.4058,86.1778,98,
16,anadyr,RU,64.7500,177.4833,80,
23,st. john's,CA,47.5649,-52.7093,87,
26,yellowknife,CA,62.4560,-114.3525,95,
29,boguchany,RU,58.3814,97.4531,94,
30,longyearbyen,SJ,78.2186,15.6401,80,
31,laojunmiao,CN,39.8333,97.7333,41,
37,magadan,RU,59.5667,150.8000,61,
47,happy valley-goose bay,CA,53.3168,-60.3315,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
labytnangi - nearest hotel: Семь лиственниц
dudinka - nearest hotel: Виктория
anadyr - nearest hotel: Гостевой дом
st. john's - nearest hotel: Alt Hotel
yellowknife - nearest hotel: No hotel found
boguchany - nearest hotel: Полет
longyearbyen - nearest hotel: The Vault
laojunmiao - nearest hotel: No hotel found
magadan - nearest hotel: Гостиница «ВМ-Центр»
happy valley-goose bay - nearest hotel: Hotel North Two
talnakh - nearest hotel: Талнах
ust-nera - nearest hotel: гостиница "Солнечная"
targu secuiesc - nearest hotel: Casa Zoltan
kokuy - nearest hotel: Гостиница
lensk - nearest hotel: No hotel found
iqaluit - nearest hotel: Frobisher Inn
baotou - nearest hotel: 包头宾馆
thompson - nearest hotel: Thompson Inn
bilibino - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
suntar - nearest hotel: No hotel found
iskateley - nearest hotel: Авантаж
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
severo-yeniseyskiy - nearest hotel: Актолик
petropavlo

,City,Country,Lat,Lng,Humidity,Hotel Name
11,labytnangi,RU,66.6572,66.4183,88,Семь лиственниц
12,dudinka,RU,69.4058,86.1778,98,Виктория
16,anadyr,RU,64.7500,177.4833,80,Гостевой дом
23,st. john's,CA,47.5649,-52.7093,87,Alt Hotel
26,yellowknife,CA,62.4560,-114.3525,95,No hotel found
...,...,...,...,...,...,...
494,susuman,RU,62.7833,148.1667,97,No hotel found
525,kirensk,RU,57.7853,108.1119,91,Ивушка
533,ola,RU,59.5833,151.2833,63,No hotel found
547,novyy urgal,RU,51.0736,132.5642,82,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng",
                                  "Lat",
                                  geo = True,
                                  color = "City",
                                  alpha = 0.8,
                                  size = "Humidity",
                                  tiles = "OSM",
                                  hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)